In [1]:
import torch
import numpy as np
import pandas as pd
import os

os.chdir("../")

%load_ext autoreload
%autoreload 2

In [5]:
from src.models.modules.encoders.swin_mae import TSwinMAE


swin3d = TSwinMAE(128, window_size=8, num_frames=4)
swin3d

TSwinMAE(
  (encoder): TSwinTransformer(
    (patch_embed): PatchEmbed3D(
      (proj): Conv3d(3, 96, kernel_size=(1, 4, 4), stride=(1, 4, 4), bias=False)
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (temporal_embed): TemporalEncoder()
    (pos_drop): Dropout(p=0.0, inplace=False)
    (tempo_layer): Linear(in_features=384, out_features=96, bias=True)
    (blocks): Sequential(
      (0): SwinTransformerStage(
        (blocks): Sequential(
          (0): SwinTransformerV2Block(
            (attn): WindowAttention(
              (cpb_mlp): Sequential(
                (0): Linear(in_features=2, out_features=512, bias=True)
                (1): ReLU(inplace=True)
                (2): Linear(in_features=512, out_features=3, bias=False)
              )
              (qkv): Linear(in_features=96, out_features=288, bias=False)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
        

In [9]:
test_input = torch.randn(1, 3, 3, 128, 128)
temp_coors = torch.tensor([[12, 256, 330]], dtype=torch.float32).unsqueeze(-1)
out = swin3d.forward_decoder(test_input, temp_coors, None)
#[x.shape for x in out]
out.shape

ValueError: too many values to unpack (expected 4)

In [20]:
from src.models.modules.decoders.swin_unet import SwinUNet

swin_unet = SwinUNet(4608, 32, window_size=8)
swin_unet

SwinUNet(
  (cat_layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=4608, out_features=2304, bias=True)
      (1): LayerNorm((2304,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=2304, out_features=2304, bias=True)
      (4): Dropout(p=0.0, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=2304, out_features=1152, bias=True)
      (1): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=1152, out_features=1152, bias=True)
      (4): Dropout(p=0.0, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=1152, out_features=576, bias=True)
      (1): LayerNorm((576,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=576, out_features=576, bias=True)
      (4): Dropout(p=0.0, inplace=False)
    )
    (3): Sequential(
      (0): Linear(in_features=576, out_

In [21]:
test_input = [
    torch.randn([1, 288, 32, 32]),
    torch.randn([1, 576, 16, 16]),
    torch.randn([1, 1152, 8, 8]),
    torch.randn([1, 2304, 4, 4]),
    torch.randn([1, 4608, 2, 2])
]

out = swin_unet(test_input)
out.shape

torch.Size([1, 288, 32, 32])

In [ ]:
n_classes = 3
B = 1
H = 4
W = 4

targets = torch.randn(B, n_classes, H, W)
logits = torch.randn(B, n_classes, H, W)

targets = torch.argmax(targets, dim=1)
logits = torch.argmax(logits, dim=1)

target_masks = torch.nn.functional.one_hot(targets, num_classes=n_classes).permute(0, 3, 1, 2)
logits_masks = torch.nn.functional.one_hot(logits, num_classes=n_classes).permute(0, 3, 1, 2)

target_points = torch.nonzero(target_masks).float()
logits_points = torch.nonzero(logits_masks).float()

i = torch.arange(B)
j = torch.arange(n_classes)
grid_i, grid_j = torch.meshgrid(i, j, indexing='ij')
coords = torch.stack([grid_i, grid_j], dim=-1).view(-1, 2) 

RuntimeError: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 3

In [2]:
from src.pytorch.losses import DCEDLoss

criterion = DCEDLoss()
criterion

DCEDLoss()

In [4]:
test_input = torch.randn(2, 19, 128, 128)
test_target = torch.randint(0, 19, (2, 128, 128))
output = criterion(test_input, test_target)
print(output)

tensor(2.1716)


In [12]:
from src.models.modules import swin

unet = swin.SwinV2Unet(img_size=128, in_channels=40, window_size=8)
unet

SwinV2Unet(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(40, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): SwinTransformerV2Stage(
      (blocks): Sequential(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (drop_path1):

In [13]:
test_input = torch.randn(1, 40, 128, 128)
test_output = unet(test_input)
test_output.shape

Layer 0: torch.Size([1, 32, 32, 96])
Layer 1: torch.Size([1, 16, 16, 192])
Layer 2: torch.Size([1, 8, 8, 384])
Layer 3: torch.Size([1, 4, 4, 768])
Final Layer: torch.Size([1, 4, 4, 768])
Layer 0: torch.Size([1, 4, 4, 768])
Layer 1: torch.Size([1, 8, 8, 384])
Layer 2: torch.Size([1, 16, 16, 192])
Layer 3: torch.Size([1, 32, 32, 96])
Final Layer: torch.Size([1, 32, 32, 96])


torch.Size([1, 2, 128, 128])

In [6]:
from src.models.modules import swin

swin_mae = swin.SwinMAE(img_size=128, in_channels=40, window_size=8)
swin_mae

SwinMAE(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(40, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): SwinTransformerV2Stage(
      (blocks): Sequential(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (drop_path1): Id

In [7]:
test_input = torch.randn(1, 40, 128, 128)
test_output = swin_mae(test_input)
test_output.shape

torch.Size([1, 1024, 640])

In [ ]:
from src.models.modules.functional import patchify, unpatchify

test_input = torch.randn(1, 40, 128, 128)
test_output = patchify(test_input, 4)
test_output.shape

torch.Size([1, 1024, 640])

In [2]:
def window_masking(
        x: torch.Tensor,
        r: int = 4,
        remove: bool = False,
        mask_len_sparse: bool = False
        ):
        """
        The new masking method, masking the adjacent r*r number of patches together

        Optional whether to remove the mask patch,
        if so, the return value returns one more sparse_restore for restoring the order to x

        Optionally, the returned mask index is sparse length or original length,
        which corresponds to the different size choices of the decoder when restoring the image

        x: [N, L, D]
        r: There are r*r patches in a window
        remove: Whether to remove the mask patch
        mask_len_sparse: Whether the returned mask length is a sparse short length
        """
        B, L, D = x.shape
        assert int(L ** 0.5 / r) == L ** 0.5 / r
        d = int(L ** 0.5 // r)

        noise = torch.rand(B, d ** 2, device=x.device)
        sparse_shuffle = torch.argsort(noise, dim=1)
        sparse_restore = torch.argsort(sparse_shuffle, dim=1)
        sparse_keep = sparse_shuffle[:, :int(d ** 2 * (1 - 0.75))]

        index_keep_part = torch.div(sparse_keep, d, rounding_mode='floor') * d * r ** 2 + sparse_keep % d * r
        index_keep = index_keep_part
        for i in range(r):
            for j in range(r):
                if i == 0 and j == 0:
                    continue
                index_keep = torch.cat([index_keep, index_keep_part + int(L ** 0.5) * i + j], dim=1)

        index_all = np.expand_dims(range(L), axis=0).repeat(B, axis=0) 
        index_mask = np.zeros([B, int(L - index_keep.shape[-1])], dtype=np.int32) 
        for i in range(B):
            # get the difference between sets
            index_mask[i] = np.setdiff1d(index_all[i], index_keep.cpu().numpy()[i], assume_unique=True)
        index_mask = torch.tensor(index_mask, device=x.device)

        index_shuffle = torch.cat([index_keep, index_mask], dim=1)
        index_restore = torch.argsort(index_shuffle, dim=1)

        mask = torch.ones([B, L], device=x.device)
        mask[:, :index_keep.shape[-1]] = 0
        mask = torch.gather(mask, dim=1, index=index_restore)

        
        x_masked = torch.clone(x)
        mask_token = torch.zeros((1, 1, D))
        for i in range(B):
            x_masked[i, index_mask.cpu().numpy()[i, :], :] = mask_token
        return x_masked, mask

In [ ]:
from src.models.modules.functional import patchify, unpatchify

test_input = patchify(torch.randn(2, 3, 64, 64), 4)
print(test_input.shape)
window_masking(test_input)[1].reshape(16, 4, 4).permute(1, 0, 2)

torch.Size([2, 256, 48])


RuntimeError: shape '[16, 4, 4]' is invalid for input of size 512

In [148]:
import torch

test_input = torch.randn(2, 64, 3)
B, L, C = test_input.shape
K = int(L ** 0.5)
r = 4
d = K // r
noise = torch.rand(B, d ** 2)
sparse_shuffle = torch.argsort(noise, dim=1)
sparse_keep = sparse_shuffle[:, :int(d ** 2 * (1 - 0.75))]
sparse_mask = sparse_shuffle[:, int(d ** 2 * (1 - 0.75)):]

indices_ = torch.arange(K * K).expand(B, -1)
indices = indices_.clone().view(B, d, r, d, r)
indices = indices.permute(0, 1, 3, 2, 4).reshape(B, d * d, r * r)
keep = torch.gather(
    indices,
    dim=1,
    index=sparse_keep.unsqueeze(-1).expand(-1, -1, r ** 2)
    ).flatten(1)
masked = torch.scatter(test_input, 1, keep.unsqueeze(-1).expand(-1, -1, C), torch.zeros(B, keep.shape[1], C))
masked.reshape(B, K, K, C)

tensor([[[[-1.5530e-01, -1.3606e+00,  5.0389e-02],
          [ 1.4056e-01, -1.0229e-01,  3.2327e-01],
          [ 1.7113e+00, -1.7365e+00, -3.9060e-01],
          [ 1.5702e+00,  9.4257e-01,  3.9562e-01],
          [-5.7084e-01, -1.0685e+00, -9.0075e-01],
          [ 1.2044e+00,  1.0893e+00,  3.9179e-01],
          [ 6.4319e-01,  2.2341e-01,  4.9420e-01],
          [-1.1133e+00, -8.5687e-02,  3.0123e-01]],

         [[-5.8830e-01,  8.7303e-01, -1.2164e+00],
          [ 2.8652e-01, -1.3424e+00,  2.1778e-01],
          [ 7.7639e-01, -1.8902e-01, -7.1140e-01],
          [-6.5874e-01,  1.0929e+00, -2.0271e+00],
          [-8.3194e-01,  1.0060e-01, -9.6272e-01],
          [ 1.0728e+00,  1.0972e+00,  7.7309e-01],
          [-8.2285e-01, -6.1732e-02,  7.9579e-01],
          [ 6.8304e-01, -2.6427e-01, -1.4681e+00]],

         [[-7.7316e-01, -8.2876e-01,  7.5420e-01],
          [-9.4114e-01,  5.4316e-01, -1.1600e+00],
          [-7.1542e-01, -2.5535e-01, -5.0733e-02],
          [ 7.4908e-01,  1.